In [5]:
%pip install fasttext nltk


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import nltk
# Configure SSL to bypass certificate verification
import ssl
import urllib.request

ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

urllib.request.install_opener(urllib.request.build_opener(urllib.request.HTTPSHandler(context=ssl_context)))

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chakrabortyb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chakrabortyb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/chakrabortyb/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [2]:
import urllib.request
import zipfile
import os

datasets_dir = "."

# download text8
url = "http://mattmahoney.net/dc/text8.zip"
zip_path = os.path.join(datasets_dir, "text8.zip")
text8_path = os.path.join(datasets_dir, "text8")

if not os.path.exists(text8_path):
    urllib.request.urlretrieve(url, zip_path)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(datasets_dir)

In [3]:
with open(text8_path, 'r') as f:
    text8_data = f.read()

# create a smaller subset (200k characters)
text8_200k_path = os.path.join(datasets_dir, "text8_200k.txt")
text8_200k_data = text8_data[:200000].split()

if not os.path.exists(text8_200k_path):
    with open(text8_200k_path, 'w') as f:
        for token in text8_200k_data:
            f.write(token + "\n")

In [4]:
# process the full text8 dataset
text8_full_path = os.path.join(datasets_dir, "text8.txt")

if not os.path.exists(text8_full_path):
    with open(text8_full_path, 'w') as f:
        for token in text8_data.split():
            f.write(token + "\n")

In [8]:
import fasttext
import os

model = fasttext.train_unsupervised(
    'text8_200k.txt',       # Input file
    model='skipgram',       
    dim=50,                 # Embedding dimension
    epoch=30,               # Number of training iterations
    minCount=1,             # Minimum word frequency
    minn=3,                 # Minimum character n-gram length
    maxn=6                  # Maximum character n-gram length
)

embeddings_path = os.path.join("..", "embeddings", "fasttext_embeddings.bin")
model.save_model(embeddings_path)

Read 0M words
Number of words:  6064
Number of labels: 0
Progress: 100.0% words/sec/thread: 1331696 lr:  0.000000 avg.loss:  3.687443 ETA:   0h 0m 0s


In [9]:
model = fasttext.train_unsupervised(
    'text8.txt',            # Input file
    model='skipgram',       
    dim=50,                 # Embedding dimension
    epoch=20,               # Number of training iterations
    minCount=1,             # Minimum word frequency
    minn=3,                 # Minimum character n-gram length
    maxn=6                  # Maximum character n-gram length
)

embeddings_path = os.path.join("..", "embeddings", "fasttext_full_embeddings.bin")
model.save_model(embeddings_path)

Read 34M words
Number of words:  253855
Number of labels: 0
Progress: 100.0% words/sec/thread: 6219384 lr:  0.000000 avg.loss:  1.293655 ETA:   0h 0m 0s 38.2% words/sec/thread: 6262430 lr:  0.030912 avg.loss:  1.551671 ETA:   0h 0m 4s


In [10]:
def get_word_embeddings(model):
    king_vector = model.get_word_vector('king')
    print(f"Vector for 'king': {king_vector[:5]}...")
    print(f"Vector shape: {king_vector.shape}")
    
    kingdom_vector = model.get_word_vector('kingdom')
    print(f"Vector for 'kingdom' (OOV): {kingdom_vector[:5]}...")
    
    return king_vector, kingdom_vector

king_vec, kingdom_vec = get_word_embeddings(model)

Vector for 'king': [-2.6219287   7.4567804  -5.975464   -2.2195573   0.07463519]...
Vector shape: (50,)
Vector for 'kingdom' (OOV): [-0.90501404  2.742808   -2.1977556  -0.8197536  -0.03171865]...


In [1]:
import csv
import re

def extract_tokens(filename):
    # Remove extension
    name = filename.rsplit('.', 1)[0] if '.' in filename else filename
    
    # Split on common delimiters: underscores, hyphens, spaces, numbers
    tokens = re.split(r'[_\-\s\d]+', name)
    
    # Filter out empty tokens and convert to lowercase
    tokens = [token.lower() for token in tokens if token.strip()]
    
    return tokens


input_file = 'filename_training_dataset_v2.csv'
output_file = 'filename_training_preprocessed.txt'

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    reader = csv.DictReader(infile)
    
    for row in reader:
        filename = row['filename']
        tokens = extract_tokens(filename)
        
        if tokens:  # Only write if we have tokens
            outfile.write(' '.join(tokens) + '\n')

print(f"Preprocessed {input_file} -> {output_file}")

Preprocessed filename_training_dataset_v2.csv -> filename_training_preprocessed.txt


In [4]:
import fasttext
import os

model = fasttext.train_unsupervised(
    output_file,            # Input file
    model='skipgram',       
    dim=50,                 # Embedding dimension
    epoch=10,               # Number of training iterations
    minCount=1,             # Minimum word frequency
    minn=3,                 # Minimum character n-gram length
    maxn=6                  # Maximum character n-gram length
)

embeddings_path = os.path.join("..", "embeddings", "fasttext_domain_embeddings.bin")
model.save_model(embeddings_path)

Read 0M words
Number of words:  184
Number of labels: 0
Progress: 100.0% words/sec/thread:  936167 lr:  0.000000 avg.loss:  2.760305 ETA:   0h 0m 0s


In [5]:
budget_vector = model.get_word_vector('budget')
print(f"Vector for 'budget': {budget_vector[:5]}...")
print(f"Vector shape: {budget_vector.shape}")

presentation_vector = model.get_word_vector('kingdom')
print(f"Vector for 'kingdom' (OOV): {presentation_vector[:5]}...")

Vector for 'budget': [-0.03002916  0.03840026 -0.02172516 -0.00011982  0.02104709]...
Vector shape: (50,)
Vector for 'kingdom' (OOV): [-0.04803699  0.06675068 -0.04020472 -0.00123968  0.03496922]...
